In [5]:
! pip install pandas
! pip install numpy
! pip install scikit-learn
! pip install matplotlib
! pip install joblib
! pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 124.9 MB/s eta 0:00:00


In [6]:
!pip -q install pandas numpy scikit-learn joblib streamlit cloudflared


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 2.1 MB/s eta 0:00:00


In [7]:
%%writefile /content/smart_recommender.py
import os, math, random
import numpy as np, pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import joblib

SEED=42
random.seed(SEED); np.random.seed(SEED)

def build_synth(n_users=600,n_items=350,minr=10,maxr=25):
    users=[f"U{u}" for u in range(1,n_users+1)]
    items=[f"P{i}" for i in range(1,n_items+1)]
    cats=['Electronics','Books','Home','Toys','Beauty','Sports','Clothing','Grocery']
    rows=[]
    for u in users:
        k=np.random.randint(minr,maxr+1)
        rated=np.random.choice(items,size=k,replace=False)
        for it in rated:
            r=np.clip(np.random.normal(3.6,1.0),1,5)
            r=round(r*2)/2.0
            rows.append((u,it,r))
    ratings=pd.DataFrame(rows,columns=['userId','productId','rating'])
    kw={'Electronics':['battery','wireless','bluetooth','USB','portable','charger'],
        'Books':['story','novel','guide','history','author','learn'],
        'Home':['kitchen','durable','design','compact','decor','clean'],
        'Toys':['kids','fun','safe','interactive','educational','colorful'],
        'Beauty':['gentle','skin','organic','scent','serum','moisturizer'],
        'Sports':['fitness','outdoor','durable','training','performance','comfort'],
        'Clothing':['fabric','comfortable','casual','size','style','soft'],
        'Grocery':['fresh','organic','snack','ingredients','package','tasty']}
    metas=[]
    for i,p in enumerate(items,1):
        c=random.choice(cats); title=f"{c} Product {i}"
        desc=" ".join(np.random.choice(kw[c],size=6,replace=True))
        metas.append({'productId':p,'title':title,'category':c,'description':f"{title}. {desc}. High quality and good value."})
    products=pd.DataFrame(metas)
    return ratings,products

def preprocess(df):
    df=df.drop_duplicates().dropna(subset=['userId','productId','rating']).reset_index(drop=True)
    df['userId']=df['userId'].astype(str); df['productId']=df['productId'].astype(str)
    df['rating']=pd.to_numeric(df['rating'],errors='coerce').astype(float)
    return df

def add_implicit(df):
    mult=np.random.uniform(0.8,1.4,size=len(df))
    df=df.copy()
    df['implicit_score']=(df['rating']/5.0)*mult
    return df

def train_cf(train_df,n_components=30):
    users=sorted(train_df['userId'].unique())
    items=sorted(train_df['productId'].unique())
    pivot=train_df.pivot_table(index='userId',columns='productId',values='rating').reindex(index=users,columns=items)
    filled=pivot.copy()
    umean=pivot.mean(axis=1); gmean=train_df['rating'].mean()
    for u in users:
        filled.loc[u]=filled.loc[u].fillna(umean.loc[u] if not np.isnan(umean.loc[u]) else gmean)
    svd=TruncatedSVD(n_components=n_components,random_state=SEED)
    U=svd.fit_transform(filled.values); V=svd.components_.T
    pred=np.dot(U,V.T)
    pred_df=pd.DataFrame(pred,index=users,columns=items)
    return svd,pred_df,filled

def train_cbf(products,max_features=2000):
    vec=TfidfVectorizer(max_features=max_features,stop_words='english')
    X=vec.fit_transform(products['description'].fillna(products['title'].fillna('')))
    return vec,X

def evaluate_rmse(pred_df,test_df):
    y_true=[]; y_pred=[]
    for _,r in test_df.iterrows():
        u,p,rt=r['userId'],r['productId'],r['rating']
        if (u in pred_df.index) and (p in pred_df.columns):
            y_true.append(rt); y_pred.append(pred_df.loc[u,p])
    return mean_squared_error(y_true,y_pred,squared=False) if y_true else None

def precision_at_k(pred_df,train_df,test_df,k=5,thr=4.0):
    users=test_df['userId'].unique(); precs=[]
    for u in users:
        if u not in pred_df.index: continue
        train_items=set(train_df[train_df['userId']==u]['productId'])
        cand=[i for i in pred_df.columns if i not in train_items]
        if not cand: continue
        top=sorted([(i,pred_df.loc[u,i]) for i in cand],key=lambda x:x[1],reverse=True)[:k]
        top_ids=[i for i,_ in top]
        relevant=set(test_df[(test_df['userId']==u)&(test_df['rating']>=thr)]['productId'])
        if not top_ids: continue
        prec=len([i for i in top_ids if i in relevant])/len(top_ids)
        precs.append(prec)
    return float(np.mean(precs)) if precs else 0.0

def train_and_save():
    ratings,products=build_synth()
    ratings=preprocess(ratings)
    ratings=add_implicit(ratings)
    train_df,test_df=train_test_split(ratings,test_size=0.2,random_state=SEED)
    svd,pred_df,filled=train_cf(train_df,30)
    vec,X=train_cbf(products)
    rmse=evaluate_rmse(pred_df,test_df)
    p5=precision_at_k(pred_df,train_df,test_df,5,4.0)
    print(f"RMSE: {rmse:.4f} | Precision@5: {p5:.4f}")
    out="/content/recommender_app/models"; os.makedirs(out,exist_ok=True)
    joblib.dump(pred_df, f"{out}/pred_matrix_df.pkl")
    joblib.dump(svd, f"{out}/truncated_svd.pkl")
    joblib.dump(vec, f"{out}/tfidf_vectorizer.pkl")
    joblib.dump(X, f"{out}/tfidf_matrix.pkl")
    products.to_pickle(f"{out}/products_df.pkl")
    train_df.to_pickle(f"{out}/train_ratings_df.pkl")
    print("Artifacts saved to", out)

if __name__=="__main__":
    train_and_save()


Writing /content/smart_recommender.py


In [8]:
%%bash
python - <<'PY'
from pathlib import Path
p = Path("/content/smart_recommender.py")
if not p.exists():
    print("ERROR: /content/smart_recommender.py not found. Make sure file path is correct.")
    raise SystemExit(1)

text = p.read_text()

old_snip = """
def evaluate_rmse(pred_df,test_df):
    y_true=[]; y_pred=[]
    for _,r in test_df.iterrows():
        u,p,rt=r['userId'],r['productId'],r['rating']
        if (u in pred_df.index) and (p in pred_df.columns):
            y_true.append(rt); y_pred.append(pred_df.loc[u,p])
    return mean_squared_error(y_true,y_pred,squared=False) if y_true else None
"""

new_snip = """
import numpy as np
def evaluate_rmse(pred_df,test_df):
    # compute RMSE without relying on sklearn's 'squared' parameter for compatibility
    y_true = []
    y_pred = []
    for _,r in test_df.iterrows():
        u = r['userId']; p = r['productId']; rt = r['rating']
        if (u in pred_df.index) and (p in pred_df.columns):
            y_true.append(rt)
            y_pred.append(pred_df.loc[u, p])
    if len(y_true) == 0:
        return None
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    mse = np.mean((y_true - y_pred) ** 2)
    return float(np.sqrt(mse))
"""

if old_snip in text:
    text = text.replace(old_snip, new_snip)
    p.write_text(text)
    print("Patched evaluate_rmse in /content/smart_recommender.py")
else:
    # fallback: try to insert function by searching for function name
    import re
    if re.search(r"def evaluate_rmse\(", text):
        text = re.sub(r"def evaluate_rmse\\([\\s\\S]*?\\)\\n\\s*return[\\s\\S]*?\\n", new_snip + "\n", text, flags=re.MULTILINE)
        p.write_text(text)
        print("Replaced evaluate_rmse via regex in /content/smart_recommender.py")
    else:
        print("Could not find evaluate_rmse function. Please open the file and replace the function with the new version manually.")
PY


Patched evaluate_rmse in /content/smart_recommender.py


In [9]:
!python /content/smart_recommender.py


RMSE: 0.9577 | Precision@5: 0.0045
Artifacts saved to /content/recommender_app/models


In [10]:
%%writefile /content/streamlit_recommender_app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity

@st.cache_resource
def load_artifacts():
    pred_matrix_df = joblib.load("/content/recommender_app/models/pred_matrix_df.pkl")
    svd = joblib.load("/content/recommender_app/models/truncated_svd.pkl")
    tfidf_vectorizer = joblib.load("/content/recommender_app/models/tfidf_vectorizer.pkl")
    tfidf_matrix = joblib.load("/content/recommender_app/models/tfidf_matrix.pkl")
    products_df = pd.read_pickle("/content/recommender_app/models/products_df.pkl")
    ratings_df = pd.read_pickle("/content/recommender_app/models/train_ratings_df.pkl")
    return pred_matrix_df, tfidf_vectorizer, tfidf_matrix, products_df, ratings_df

pred_matrix_df, tfidf_vectorizer, tfidf_matrix, products_df, ratings_df = load_artifacts()

CATEGORY_TO_IMAGE = {
    "Electronics": "https://source.unsplash.com/800x520/?electronics,gadgets",
    "Books": "https://source.unsplash.com/800x520/?books,reading",
    "Home": "https://source.unsplash.com/800x520/?home,interior",
    "Toys": "https://source.unsplash.com/800x520/?toys,kids",
    "Beauty": "https://source.unsplash.com/800x520/?beauty,cosmetics",
    "Sports": "https://source.unsplash.com/800x520/?sports,fitness",
    "Clothing": "https://source.unsplash.com/800x520/?clothing,fashion",
    "Grocery": "https://source.unsplash.com/800x520/?grocery,food",
}
def img_for(cat): return CATEGORY_TO_IMAGE.get(cat, "https://source.unsplash.com/800x520/?product")

PROD_TO_IDX = {pid: idx for idx, pid in enumerate(products_df["productId"])}
POPULAR_ORDER = list(ratings_df["productId"].value_counts().index)

def dynamic_weights(num_ratings, min_r=5, max_r=40):
    if num_ratings <= min_r: a_cf = 0.25
    else: a_cf = 0.25 + 0.65 * min(num_ratings - min_r, max_r - min_r) / (max_r - min_r)
    return a_cf, 1.0 - a_cf

def build_explanations(user_id, pid, cf_raw, cbf_score):
    reasons=[]
    cf_norm=(cf_raw-1)/4 if cf_raw is not None else 0.0
    if cf_norm>0.5: reasons.append("Users with tastes like yours rated this highly.")
    liked = ratings_df[(ratings_df.userId==user_id) & (ratings_df.rating>=4.0)]['productId'].tolist()
    if cbf_score>0.12 and liked:
        liked_idxs=[PROD_TO_IDX[l] for l in liked if l in PROD_TO_IDX]
        if pid in PROD_TO_IDX and liked_idxs:
            p_idx=PROD_TO_IDX[pid]
            sims=cosine_similarity(tfidf_matrix[p_idx], tfidf_matrix[liked_idxs]).flatten()
            best_idx=liked_idxs[int(np.argmax(sims))]
            best_title=products_df.iloc[best_idx]['title']
            reasons.append(f"Similar to what you liked: '{best_title}'.")
        else:
            reasons.append("Shares features with items you liked.")
    if pid in POPULAR_ORDER:
        rnk=POPULAR_ORDER.index(pid)+1
        if rnk<=20: reasons.append(f"Popular choice (top {rnk} most-rated).")
    if not reasons: reasons.append("Recommended by hybrid model signals.")
    return reasons

def recommend(user_id, n=5):
    all_p = products_df['productId'].tolist()
    seen = ratings_df[ratings_df.userId==user_id]['productId'].tolist()
    cand = [p for p in all_p if p not in seen]
    num_r = len(ratings_df[ratings_df.userId==user_id])
    a_cf,a_cbf = dynamic_weights(num_r)
    rows=[]
    for p in cand:
        cf_raw = float(pred_matrix_df.loc[user_id,p]) if (user_id in pred_matrix_df.index and p in pred_matrix_df.columns) else 0.0
        cf_norm=(cf_raw-1)/4
        liked = ratings_df[(ratings_df.userId==user_id) & (ratings_df.rating>=4.0)]['productId'].tolist()
        if liked and (p in PROD_TO_IDX):
            p_idx=PROD_TO_IDX[p]
            liked_idxs=[PROD_TO_IDX[l] for l in liked if l in PROD_TO_IDX]
            cbf_sim = float(cosine_similarity(tfidf_matrix[p_idx], tfidf_matrix[liked_idxs]).flatten().mean()) if liked_idxs else 0.0
        else:
            cbf_sim=0.0
        score = a_cf*cf_norm + a_cbf*cbf_sim
        meta = products_df.loc[products_df.productId==p].iloc[0]
        rows.append({
            "productId": p,
            "title": meta["title"],
            "category": meta["category"],
            "description": meta["description"],
            "hybrid_score": float(score),
            "cf_raw": float(cf_raw),
            "cbf_sim": float(cbf_sim),
            "reasons": build_explanations(user_id,p,cf_raw,cbf_sim),
            "image_url": img_for(meta["category"]),
        })
    out = pd.DataFrame(rows).sort_values("hybrid_score", ascending=False).head(n).reset_index(drop=True)
    return out, a_cf, a_cbf

st.title("🛒 Smart Product Recommender Dashboard (Colab)")
st.caption("Weighted Hybrid: Collaborative + Content-Based with explanations")

users = sorted(ratings_df['userId'].unique())
user = st.selectbox("Select User ID", users)
n = st.slider("Number of recommendations", 3, 10, 5)

if st.button("🔍 Get Recommendations"):
    df, a_cf, a_cbf = recommend(user, n)
    st.subheader(f"🎯 Recommendations for {user}")
    st.caption(f"Weights → CF: {a_cf:.2f} | CBF: {a_cbf:.2f}")
    for _, row in df.iterrows():
        st.image(row["image_url"])
        st.markdown(f"### {row['title']}")
        st.markdown(f"*Category:* {row['category']}")
        st.markdown(f"**Hybrid Score:** {row['hybrid_score']:.3f}  |  **CF(raw):** {row['cf_raw']:.3f}  |  **CBF(sim):** {row['cbf_sim']:.3f}")
        st.markdown("**Why this?**")
        for reason in row["reasons"]:
            st.markdown(f"- {reason}")
        st.caption(row["description"][:220] + ("..." if len(row["description"])>220 else ""))
        st.divider()

st.sidebar.header("📈 How it works")
st.sidebar.markdown("- **CF** learns from similar users' ratings.\n- **CBF** compares product text (TF-IDF).\n- **Weights** adapt to how active the user is (cold-start aware).\n- Unsplash placeholders used for images; replace with your own if available.")


Overwriting /content/streamlit_recommender_app.py


In [11]:
# Start Streamlit in background
!streamlit run /content/streamlit_recommender_app.py &>/content/streamlit.log &

# Expose via Cloudflare tunnel (prints a public URL)
!nohup cloudflared tunnel --url http://localhost:8501 --no-autoupdate > /content/cf.log 2>&1 & sleep 3; grep -o 'https://.*trycloudflare.com' -m 1 /content/cf.log


https://fuzzy-steve-tablets-scientists.trycloudflare.com


In [12]:
# 1) Reinstall/ensure deps
!pip -q install cloudflared streamlit >/dev/null

# 2) Kill any stale processes
!pkill -f streamlit || true
!pkill -f cloudflared || true

# 3) Start Streamlit (headless) in background
!streamlit run /content/streamlit_recommender_app.py --server.port 8501 --server.headless true &>/content/streamlit.log &

# 4) Start Cloudflare tunnel in background
!nohup cloudflared tunnel --url http://localhost:8501 --no-autoupdate > /content/cf.log 2>&1 &

^C
^C


In [13]:
# Is Streamlit actually running?
!ps -ef | grep streamlit | grep -v grep

# Any traceback in Streamlit logs?
!sed -n '1,200p' /content/streamlit.log

# Any errors in cloudflared?
!sed -n '1,200p' /content/cf.log

root        2434       1 22 12:38 ?        00:00:00 /usr/bin/python3 /usr/local/bin/streamlit run /content/streamlit_recommender_app.py --server.port 8501 --server.headless true



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.60.96.97:8501

2025-10-13T12:38:37Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-13T12:38:37Z INF Requesting new quick 

**sentence transformers**

In [14]:
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer
import numpy as np, joblib, pandas as pd

# Load your product metadata
products_df = pd.read_pickle("/content/recommender_app/models/products_df.pkl")

# Build semantic embeddings using a pre-trained transformer
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode product descriptions (semantic understanding)
embeddings = model.encode(
    products_df["description"].fillna("").tolist(),
    show_progress_bar=True,
    batch_size=64,
    convert_to_numpy=True
)

# Save embeddings & model
np.save("/content/recommender_app/models/desc_embeddings.npy", embeddings)
joblib.dump(model, "/content/recommender_app/models/sentence_model.pkl")

print("✅ Semantic embeddings created and saved.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Semantic embeddings created and saved.


**trend scores**

In [15]:
import numpy as np, pandas as pd

ratings_df = pd.read_pickle("/content/recommender_app/models/train_ratings_df.pkl")
products_df = pd.read_pickle("/content/recommender_app/models/products_df.pkl")

# Simulate timestamps (for demo)
np.random.seed(42)
ratings_df["timestamp"] = pd.to_datetime(
    np.random.choice(pd.date_range("2024-01-01", "2025-10-01"), len(ratings_df))
)

# Compute normalized recency weights
latest = ratings_df["timestamp"].max()
ratings_df["recency_weight"] = 1 - (
    (latest - ratings_df["timestamp"]).dt.days / (latest - ratings_df["timestamp"]).dt.days.max()
)

trend = ratings_df.groupby("productId")["recency_weight"].mean()
products_df = products_df.merge(trend.rename("trend_score"), on="productId", how="left").fillna(0.5)

# Save updated product metadata
products_df.to_pickle("/content/recommender_app/models/products_df.pkl")

print("✅ Trend scores added and products_df updated.")


✅ Trend scores added and products_df updated.


In [16]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
!cloudflared --version


/usr/local/bin/cloudflared: Text file busy
cloudflared version 2025.9.1 (built 2025-09-22-13:28 UTC)


**cloudflare**

In [17]:
%%bash
# Kill any existing processes
pkill -f streamlit || true
pkill -f cloudflared || true
sleep 1

# Start Streamlit (background)
nohup streamlit run /content/streamlit_recommender_app.py --server.port 8501 --server.headless true > /content/streamlit.log 2>&1 &

# Start Cloudflare tunnel
nohup cloudflared tunnel --url http://localhost:8501 --no-autoupdate > /content/cf.log 2>&1 &

# Wait up to 30s for the public URL
echo "Waiting for Cloudflare public URL..."
for i in {1..30}; do
  URL=$(grep -o "https://[a-z0-9.-]*\\.trycloudflare\\.com" -m 1 /content/cf.log || true)
  if [ -n "$URL" ]; then
    echo ""
    echo "✅ Streamlit app is live!"
    echo "🌐 Open this Cloudflare URL in a new tab:"
    echo "$URL"
    break
  fi
  sleep 1
done

# If no URL, print logs
if [ -z "$URL" ]; then
  echo ""
  echo "⚠️ Could not detect Cloudflare URL. Showing logs:"
  tail -n 60 /content/cf.log
fi


Waiting for Cloudflare public URL...

✅ Streamlit app is live!
🌐 Open this Cloudflare URL in a new tab:
https://cell-university-entertaining-everybody.trycloudflare.com


**cloudflare + localnet**

In [37]:
%%bash
set -euo pipefail
echo "=== Unified launcher: Streamlit + Cloudflare + localtunnel ==="

# 0) safety: show files
echo "Files in /content (top):"
ls -la /content | sed -n '1,120p'
echo "----------------------------------"

# 1) install/update cloudflared binary (idempotent)
echo "Installing cloudflared..."
wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared || true
chmod +x /usr/local/bin/cloudflared || true
echo "cloudflared version:"
cloudflared --version || echo "(cloudflared not available)"

# 2) ensure node + localtunnel (npx) available
if ! command -v npx >/dev/null 2>&1; then
  echo "Installing nodejs & npm..."
  apt-get update -y >/dev/null
  apt-get install -y nodejs npm >/dev/null
fi
echo "Installing localtunnel (may be no-op)..."
npm install -g localtunnel >/dev/null 2>&1 || true

# 3) kill stale processes
echo "Killing old streamlit/cloudflared/localtunnel processes (if any)..."
pkill -f streamlit || true
pkill -f cloudflared || true
pkill -f localtunnel || true
pkill -f lt || true
sleep 1

# 4) confirm Streamlit app exists
APP="/content/streamlit_recommender_app.py"
if [ ! -f "$APP" ]; then
  echo "ERROR: Streamlit app not found at $APP"
  echo "Make sure file exists (run training cell or create the app). Aborting."
  exit 1
fi

# 5) start Streamlit (background)
echo "Starting Streamlit..."
nohup streamlit run "$APP" --server.port 8501 --server.headless true > /content/streamlit.log 2>&1 &
STREAMLIT_PID=$!
sleep 1
echo "Streamlit pid: $STREAMLIT_PID"
echo "Tail of streamlit.log:"
tail -n 40 /content/streamlit.log || true

# 6) start cloudflared tunnel (background) -> cf.log
echo "Starting cloudflared tunnel..."
nohup cloudflared tunnel --url http://localhost:8501 --no-autoupdate > /content/cf.log 2>&1 &
CLOUD_PID=$!
sleep 1

# 7) start localtunnel (background) -> lt.log
echo "Starting localtunnel (npx localtunnel)..."
rm -f /content/lt.log
nohup npx localtunnel --port 8501 > /content/lt.log 2>&1 &
LT_PID=$!
sleep 1

# 8) wait for URLs (prefer cloudflare, but accept either). Timeout ~40s
echo "Waiting up to 40s for a public URL (Cloudflare preferred)..."
CLOUD_URL=""
LT_URL=""
for i in {1..40}; do
  if [ -z "$CLOUD_URL" ]; then
    CLOUD_URL=$(grep -o "https://[a-z0-9.-]*\\.trycloudflare\\.com" -m 1 /content/cf.log || true)
  fi
  if [ -z "$LT_URL" ]; then
    LT_URL=$(grep -o "https://[a-z0-9.-]*\\.loca\\.lt" -m 1 /content/lt.log || true)
  fi
  if [ -n "$CLOUD_URL" ] || [ -n "$LT_URL" ]; then
    break
  fi
  sleep 1
done

# 9) print results & helpful logs
if [ -n "$CLOUD_URL" ]; then
  echo ""
  echo "✅ Cloudflare tunnel URL detected:"
  echo "$CLOUD_URL"
  echo "Open it in your browser. If you hit DNS issues, check the logs below."
elif [ -n "$LT_URL" ]; then
  echo ""
  echo "✅ localtunnel URL detected:"
  echo "$LT_URL"
  echo "Open it in your browser. You may be asked for the loca.lt tunnel password on first visit."
else
  echo ""
  echo "⚠️ No public URL detected within timeout."
  echo "---- last lines of cloudflared log (/content/cf.log) ----"
  tail -n 80 /content/cf.log || true
  echo "---- last lines of localtunnel log (/content/lt.log) ----"
  tail -n 80 /content/lt.log || true
  echo "---- last lines of streamlit log (/content/streamlit.log) ----"
  tail -n 80 /content/streamlit.log || true
  echo "You can re-run this cell to try again after fixing errors."
  exit 1
fi

echo ""
echo "Streamlit process PID: $STREAMLIT_PID"
echo "cloudflared PID: $CLOUD_PID (if cloudflared started)"
echo "localtunnel PID: $LT_PID (if localtunnel started)"
echo ""
echo "If you need the tunnel password for loca.lt, run: curl -s https://loca.lt/mytunnelpassword"
echo "Logs: /content/streamlit.log, /content/cf.log, /content/lt.log"
echo "Done."


=== Unified launcher: Streamlit + Cloudflare + localtunnel ===
Files in /content (top):
total 72
drwxr-xr-x 1 root root  4096 Oct 13 12:40 .
drwxr-xr-x 1 root root  4096 Oct 13 12:29 ..
-rw-r--r-- 1 root root  3021 Oct 13 13:01 cf.log
drwxr-xr-x 4 root root  4096 Oct  9 13:35 .config
-rw-r--r-- 1 root root    45 Oct 13 12:40 lt.log
drwxr-xr-x 3 root root  4096 Oct 13 12:38 recommender_app
drwxr-xr-x 1 root root  4096 Oct  9 13:36 sample_data
-rw-r--r-- 1 root root  5542 Oct 13 12:38 smart_recommender.py
-rw-r--r-- 1 root root 16797 Oct 13 13:04 streamlit.log
-rw-r--r-- 1 root root 13426 Oct 13 13:01 streamlit_recommender_app.py
----------------------------------
Installing cloudflared...
cloudflared version:
cloudflared version 2025.9.1 (built 2025-09-22-13:28 UTC)
Installing localtunnel (may be no-op)...
Killing old streamlit/cloudflared/localtunnel processes (if any)...
Starting Streamlit...
Streamlit pid: 14404
Tail of streamlit.log:



  You can now view your Streamlit app in your 

/usr/local/bin/cloudflared: Text file busy


**Add a real time user feedback learning loop**

In [41]:
%%bash
cat > /content/streamlit_recommender_app.py <<'PY'
import streamlit as st
import pandas as pd
import numpy as np
import joblib, os, tempfile
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import hashlib

# ===============================
# 1️⃣ Load Artifacts
# ===============================
@st.cache_resource
def load_artifacts():
    pred_matrix_df = joblib.load("/content/recommender_app/models/pred_matrix_df.pkl")
    svd = joblib.load("/content/recommender_app/models/truncated_svd.pkl")
    tfidf_vectorizer = joblib.load("/content/recommender_app/models/tfidf_vectorizer.pkl")
    tfidf_matrix = joblib.load("/content/recommender_app/models/tfidf_matrix.pkl")
    products_df = pd.read_pickle("/content/recommender_app/models/products_df.pkl")
    ratings_df = pd.read_pickle("/content/recommender_app/models/train_ratings_df.pkl")
    return pred_matrix_df, tfidf_vectorizer, tfidf_matrix, products_df, ratings_df

pred_matrix_df, tfidf_vectorizer, tfidf_matrix, products_df, ratings_df = load_artifacts()

# ===============================
# 2️⃣ Helper Mappings & Utilities
# ===============================
def get_unique_product_image(product_id, title, category):
    """
    Generate unique image URL for each product
    Uses Picsum Photos which is more reliable than Unsplash random
    """
    try:
        # Create a unique but consistent seed from productId
        seed = int(hashlib.md5(str(product_id).encode()).hexdigest()[:8], 16) % 1000

        # Use Lorem Picsum for reliable, unique images
        # Format: https://picsum.photos/seed/{seed}/800/600
        image_url = f"https://picsum.photos/seed/{product_id}_{seed}/800/600"

        return image_url
    except Exception as e:
        # Fallback to a generic image
        return "https://picsum.photos/800/600?random=1"

PROD_TO_IDX = {pid: idx for idx, pid in enumerate(products_df["productId"])}
POPULAR_ORDER = list(ratings_df["productId"].value_counts().index)

# ===============================
# 3️⃣ Adaptive Weighting
# ===============================
def dynamic_weights(num_ratings, min_r=5, max_r=40):
    if num_ratings <= min_r: a_cf = 0.25
    else: a_cf = 0.25 + 0.65 * min(num_ratings - min_r, max_r - min_r) / (max_r - min_r)
    return a_cf, 1.0 - a_cf

def adjust_weights_from_feedback(user_id, base_cf=0.6, base_cbf=0.4, fb_path="/content/recommender_app/models/feedback.csv"):
    if not os.path.exists(fb_path): return base_cf, base_cbf
    fb = pd.read_csv(fb_path)
    ufb = fb[fb["userId"] == user_id]
    if ufb.empty: return base_cf, base_cbf
    mean_fb = ufb["feedback_score"].mean()
    if mean_fb >= 0.6: base_cf += 0.05
    elif mean_fb <= 0.4: base_cbf += 0.05
    s = base_cf + base_cbf
    return base_cf/s, base_cbf/s

# ===============================
# 4️⃣ Explanation Generator
# ===============================
def build_explanations(user_id, pid, cf_raw, cbf_score):
    reasons=[]
    cf_norm=(cf_raw-1)/4 if cf_raw is not None else 0.0
    if cf_norm>0.5: reasons.append("Users similar to you rated this product highly.")
    liked = ratings_df[(ratings_df.userId==user_id) & (ratings_df.rating>=4.0)]['productId'].tolist()
    if cbf_score>0.12 and liked:
        liked_idxs=[PROD_TO_IDX[l] for l in liked if l in PROD_TO_IDX]
        if pid in PROD_TO_IDX and liked_idxs:
            p_idx=PROD_TO_IDX[pid]
            sims=cosine_similarity(tfidf_matrix[p_idx], tfidf_matrix[liked_idxs]).flatten()
            best_idx=liked_idxs[int(np.argmax(sims))]
            best_title=products_df.iloc[best_idx]['title']
            reasons.append(f"Similar to your liked item: '{best_title}'.")
    if pid in POPULAR_ORDER:
        rnk=POPULAR_ORDER.index(pid)+1
        if rnk<=20: reasons.append(f"Currently trending (Top {rnk} most rated).")
    if not reasons: reasons.append("Suggested by hybrid model blend.")
    return reasons

# ===============================
# 5️⃣ Recommender Core
# ===============================
def recommend(user_id, n=5):
    all_p = products_df['productId'].tolist()
    seen = ratings_df[ratings_df.userId==user_id]['productId'].tolist()
    cand = [p for p in all_p if p not in seen]
    num_r = len(ratings_df[ratings_df.userId==user_id])
    a_cf,a_cbf = dynamic_weights(num_r)
    a_cf,a_cbf = adjust_weights_from_feedback(user_id, a_cf, a_cbf)

    rows=[]
    for p in cand:
        cf_raw = float(pred_matrix_df.loc[user_id,p]) if (user_id in pred_matrix_df.index and p in pred_matrix_df.columns) else 0.0
        cf_norm=(cf_raw-1)/4
        liked = ratings_df[(ratings_df.userId==user_id) & (ratings_df.rating>=4.0)]['productId'].tolist()
        if liked and (p in PROD_TO_IDX):
            p_idx=PROD_TO_IDX[p]
            liked_idxs=[PROD_TO_IDX[l] for l in liked if l in PROD_TO_IDX]
            cbf_sim = float(cosine_similarity(tfidf_matrix[p_idx], tfidf_matrix[liked_idxs]).flatten().mean()) if liked_idxs else 0.0
        else:
            cbf_sim=0.0
        score = a_cf*cf_norm + a_cbf*cbf_sim
        meta = products_df.loc[products_df.productId==p].iloc[0]
        rows.append({
            "productId": p,
            "title": meta["title"],
            "category": meta["category"],
            "description": meta["description"],
            "hybrid_score": float(score),
            "cf_raw": float(cf_raw),
            "cbf_sim": float(cbf_sim),
            "reasons": build_explanations(user_id,p,cf_raw,cbf_sim),
            "image_url": get_unique_product_image(p, meta["title"], meta["category"]),
        })
    out = pd.DataFrame(rows).sort_values("hybrid_score", ascending=False).head(n).reset_index(drop=True)
    return out, a_cf, a_cbf

# ===============================
# 6️⃣ Streamlit UI
# ===============================
st.set_page_config(layout="wide", page_title="Smart Product Recommender", page_icon="🛒")

# Custom CSS
st.markdown("""
    <style>
    html {
        scroll-behavior: smooth;
    }
    .stButton button {
        width: 100%;
    }
    </style>
    """, unsafe_allow_html=True)

st.title("🛒 Smart Product Recommender Dashboard")
st.caption("Hybrid CF + CBF | Feedback Adaptive | Explainable")

# Initialize session state
if 'recommendations_generated' not in st.session_state:
    st.session_state.recommendations_generated = False
if 'current_recommendations' not in st.session_state:
    st.session_state.current_recommendations = None
if 'current_user' not in st.session_state:
    st.session_state.current_user = None
if 'current_weights' not in st.session_state:
    st.session_state.current_weights = (0.6, 0.4)

users = sorted(ratings_df['userId'].unique())
user = st.selectbox("Select User ID", users)
n = st.slider("Number of recommendations", 3, 10, 5)

if st.button("🔍 Get Recommendations", type="primary"):
    with st.spinner("Generating personalized recommendations..."):
        df, a_cf, a_cbf = recommend(user, n)
        st.session_state.current_recommendations = df
        st.session_state.current_user = user
        st.session_state.current_weights = (a_cf, a_cbf)
        st.session_state.recommendations_generated = True
        st.success(f"✅ Generated {len(df)} recommendations!")

# Display recommendations if they exist
if st.session_state.recommendations_generated and st.session_state.current_recommendations is not None:
    df = st.session_state.current_recommendations
    user = st.session_state.current_user
    a_cf, a_cbf = st.session_state.current_weights

    st.subheader(f"🎯 Top {len(df)} Recommendations for User {user}")
    st.caption(f"Model Weights → CF: {a_cf:.2f} | CBF: {a_cbf:.2f}")

    st.markdown("---")

    for idx, row in df.iterrows():
        with st.container():
            col_img, col_details = st.columns([1, 2])

            with col_img:
                # Display product image
                st.image(row["image_url"], use_container_width=True)
                st.caption(f"📦 {row['category']}")

            with col_details:
                st.markdown(f"### {idx + 1}. {row['title']}")

                # Score metrics
                metric_col1, metric_col2, metric_col3 = st.columns(3)
                with metric_col1:
                    st.metric("Hybrid Score", f"{row['hybrid_score']:.3f}")
                with metric_col2:
                    st.metric("CF Score", f"{row['cf_raw']:.3f}")
                with metric_col3:
                    st.metric("CBF Similarity", f"{row['cbf_sim']:.3f}")

                # Explanation
                st.markdown("**💡 Why this recommendation?**")
                for reason in row["reasons"]:
                    st.markdown(f"- {reason}")

                # Description
                with st.expander("📝 View Full Description"):
                    st.write(row["description"])

                # FEEDBACK SECTION
                FEEDBACK_PATH = "/content/recommender_app/models/feedback.csv"
                os.makedirs(os.path.dirname(FEEDBACK_PATH), exist_ok=True)

                def append_feedback_row(row_dict):
                    header_needed = not os.path.exists(FEEDBACK_PATH)
                    df_fb = pd.DataFrame([row_dict])
                    try:
                        df_fb.to_csv(FEEDBACK_PATH, index=False, header=header_needed, mode="a")
                    except Exception:
                        with open(FEEDBACK_PATH, "a") as f:
                            if header_needed:
                                f.write(",".join(df_fb.columns.tolist()) + "\n")
                            f.write(",".join(map(str, list(row_dict.values()))) + "\n")

                pid = row["productId"]

                # State keys
                state_like_key = f"liked_{user}_{pid}"
                state_dislike_key = f"disliked_{user}_{pid}"

                # Initialize
                if state_like_key not in st.session_state:
                    st.session_state[state_like_key] = False
                if state_dislike_key not in st.session_state:
                    st.session_state[state_dislike_key] = False

                # Callbacks
                def handle_like():
                    fb = {
                        "timestamp": datetime.utcnow().isoformat(),
                        "userId": user,
                        "productId": pid,
                        "feedback_score": 1
                    }
                    append_feedback_row(fb)
                    st.session_state[state_like_key] = True
                    st.session_state[state_dislike_key] = False

                def handle_dislike():
                    fb = {
                        "timestamp": datetime.utcnow().isoformat(),
                        "userId": user,
                        "productId": pid,
                        "feedback_score": 0
                    }
                    append_feedback_row(fb)
                    st.session_state[state_dislike_key] = True
                    st.session_state[state_like_key] = False

                def handle_undo_like():
                    st.session_state[state_like_key] = False

                def handle_undo_dislike():
                    st.session_state[state_dislike_key] = False

                # Display buttons
                st.markdown("**📢 Your Feedback:**")
                fb_col1, fb_col2, fb_col3 = st.columns([1, 1, 2])

                with fb_col1:
                    if not st.session_state[state_like_key]:
                        if st.button("👍 Like", key=f"like_{pid}"):
                            handle_like()
                            st.rerun()
                    else:
                        st.success("✓ Liked!")
                        if st.button("↩️ Undo", key=f"undo_like_{pid}"):
                            handle_undo_like()
                            st.rerun()

                with fb_col2:
                    if not st.session_state[state_dislike_key]:
                        if st.button("👎 Dislike", key=f"dislike_{pid}"):
                            handle_dislike()
                            st.rerun()
                    else:
                        st.error("✗ Disliked!")
                        if st.button("↩️ Undo", key=f"undo_dislike_{pid}"):
                            handle_undo_dislike()
                            st.rerun()

                with fb_col3:
                    if st.session_state[state_like_key]:
                        st.info("💬 Great! We'll show you more like this.")
                    elif st.session_state[state_dislike_key]:
                        st.info("💬 Noted! We'll adjust your recommendations.")

            st.markdown("---")

# Sidebar
st.sidebar.header("📊 System Stats")
st.sidebar.metric("Total Products", len(products_df))
st.sidebar.metric("Total Users", len(ratings_df['userId'].unique()))
st.sidebar.metric("Total Ratings", len(ratings_df))

st.sidebar.header("📈 How It Works")
st.sidebar.markdown("""
- **CF**: Learns from similar users
- **CBF**: Uses product features
- **Dynamic Weights**: Adapts to feedback
- **Unique Images**: Each product gets different image
- **Real-time Feedback**: Like/dislike saves instantly
""")

if st.session_state.recommendations_generated:
    st.sidebar.success(f"✅ Showing {len(st.session_state.current_recommendations)} recommendations")
PY

# Restart services
pkill -f streamlit || true
pkill -f cloudflared || true
sleep 2
nohup streamlit run /content/streamlit_recommender_app.py --server.port 8501 --server.headless true > /content/streamlit.log 2>&1 &
nohup cloudflared tunnel --url http://localhost:8501 --no-autoupdate > /content/cf.log 2>&1 &
sleep 5
grep -o "https://[a-z0-9.-]*\\.trycloudflare\\.com" -m 1 /content/cf.log || true

https://sink-dependence-empire-forever.trycloudflare.com


**testing feedback**

In [56]:
import os, pandas as pd
p="/content/recommender_app/models/feedback.csv"
print("exists:", os.path.exists(p))
if os.path.exists(p):
    display(pd.read_csv(p).tail(10))



exists: True


,timestamp,userId,productId,feedback_score
16,2025-10-13T16:05:42.703139,U1,P179,1
17,2025-10-13T16:05:48.564783,U1,P179,0
18,2025-10-13T16:05:52.237286,U1,P179,1
19,2025-10-13T16:20:16.107166,U1,P289,1
20,2025-10-13T16:20:35.978976,U1,P289,0
21,2025-10-13T16:22:52.380888,U101,P5,1
22,2025-10-13T16:23:09.845694,U101,P302,1
23,2025-10-13T16:23:27.069561,U101,P302,0
24,2025-10-13T16:31:20.015610,U1,P1,1
25,2025-10-13T16:32:16.714087,U1,P1,0


**adding searchable dropdowns**

In [57]:
%%bash
cat > /content/streamlit_recommender_app.py <<'PY'
import streamlit as st
import pandas as pd
import numpy as np
import joblib, os, tempfile
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import hashlib

# ===============================
# 1️⃣ Load Artifacts
# ===============================
@st.cache_resource
def load_artifacts():
    pred_matrix_df = joblib.load("/content/recommender_app/models/pred_matrix_df.pkl")
    svd = joblib.load("/content/recommender_app/models/truncated_svd.pkl")
    tfidf_vectorizer = joblib.load("/content/recommender_app/models/tfidf_vectorizer.pkl")
    tfidf_matrix = joblib.load("/content/recommender_app/models/tfidf_matrix.pkl")
    products_df = pd.read_pickle("/content/recommender_app/models/products_df.pkl")
    ratings_df = pd.read_pickle("/content/recommender_app/models/train_ratings_df.pkl")
    return pred_matrix_df, tfidf_vectorizer, tfidf_matrix, products_df, ratings_df

pred_matrix_df, tfidf_vectorizer, tfidf_matrix, products_df, ratings_df = load_artifacts()

# ===============================
# 2️⃣ Helper Mappings & Utilities
# ===============================
PROD_TO_IDX = {pid: idx for idx, pid in enumerate(products_df["productId"])}
POPULAR_ORDER = list(ratings_df["productId"].value_counts().index)

# ===============================
# 3️⃣ Adaptive Weighting
# ===============================
def dynamic_weights(num_ratings, min_r=5, max_r=40):
    if num_ratings <= min_r: a_cf = 0.25
    else: a_cf = 0.25 + 0.65 * min(num_ratings - min_r, max_r - min_r) / (max_r - min_r)
    return a_cf, 1.0 - a_cf

def adjust_weights_from_feedback(user_id, base_cf=0.6, base_cbf=0.4, fb_path="/content/recommender_app/models/feedback.csv"):
    if not os.path.exists(fb_path): return base_cf, base_cbf
    fb = pd.read_csv(fb_path)
    ufb = fb[fb["userId"] == user_id]
    if ufb.empty: return base_cf, base_cbf
    mean_fb = ufb["feedback_score"].mean()
    if mean_fb >= 0.6: base_cf += 0.05
    elif mean_fb <= 0.4: base_cbf += 0.05
    s = base_cf + base_cbf
    return base_cf/s, base_cbf/s

# ===============================
# 4️⃣ Explanation Generator
# ===============================
def build_explanations(user_id, pid, cf_raw, cbf_score):
    reasons=[]
    cf_norm=(cf_raw-1)/4 if cf_raw is not None else 0.0
    if cf_norm>0.5: reasons.append("Users similar to you rated this product highly.")
    liked = ratings_df[(ratings_df.userId==user_id) & (ratings_df.rating>=4.0)]['productId'].tolist()
    if cbf_score>0.12 and liked:
        liked_idxs=[PROD_TO_IDX[l] for l in liked if l in PROD_TO_IDX]
        if pid in PROD_TO_IDX and liked_idxs:
            p_idx=PROD_TO_IDX[pid]
            sims=cosine_similarity(tfidf_matrix[p_idx], tfidf_matrix[liked_idxs]).flatten()
            best_idx=liked_idxs[int(np.argmax(sims))]
            best_title=products_df.iloc[best_idx]['title']
            reasons.append(f"Similar to your liked item: '{best_title}'.")
    if pid in POPULAR_ORDER:
        rnk=POPULAR_ORDER.index(pid)+1
        if rnk<=20: reasons.append(f"Currently trending (Top {rnk} most rated).")
    if not reasons: reasons.append("Suggested by hybrid model blend.")
    return reasons

# ===============================
# 5️⃣ Recommender Core - Modified for Single Product
# ===============================
def recommend_single_product(user_id, product_id):
    """Get recommendation score and details for a specific user-product pair"""
    num_r = len(ratings_df[ratings_df.userId==user_id])
    a_cf, a_cbf = dynamic_weights(num_r)
    a_cf, a_cbf = adjust_weights_from_feedback(user_id, a_cf, a_cbf)

    # Calculate scores for the specific product
    cf_raw = float(pred_matrix_df.loc[user_id, product_id]) if (user_id in pred_matrix_df.index and product_id in pred_matrix_df.columns) else 0.0
    cf_norm = (cf_raw - 1) / 4

    liked = ratings_df[(ratings_df.userId==user_id) & (ratings_df.rating>=4.0)]['productId'].tolist()
    if liked and (product_id in PROD_TO_IDX):
        p_idx = PROD_TO_IDX[product_id]
        liked_idxs = [PROD_TO_IDX[l] for l in liked if l in PROD_TO_IDX]
        cbf_sim = float(cosine_similarity(tfidf_matrix[p_idx], tfidf_matrix[liked_idxs]).flatten().mean()) if liked_idxs else 0.0
    else:
        cbf_sim = 0.0

    score = a_cf * cf_norm + a_cbf * cbf_sim
    meta = products_df.loc[products_df.productId==product_id].iloc[0]

    result = {
        "productId": product_id,
        "title": meta["title"],
        "category": meta["category"],
        "description": meta["description"],
        "hybrid_score": float(score),
        "cf_raw": float(cf_raw),
        "cbf_sim": float(cbf_sim),
        "reasons": build_explanations(user_id, product_id, cf_raw, cbf_sim),
    }

    return result, a_cf, a_cbf

# ===============================
# 6️⃣ Streamlit UI
# ===============================
st.set_page_config(layout="wide", page_title="Smart Product Recommender", page_icon="🛒")

# Custom CSS
st.markdown("""
    <style>
    html {
        scroll-behavior: smooth;
    }
    .stButton button {
        width: 100%;
    }
    </style>
    """, unsafe_allow_html=True)

st.title("🛒 Smart Product Recommender Dashboard")
st.caption("Hybrid CF + CBF | Feedback Adaptive | Explainable")

# Initialize session state
if 'recommendation_generated' not in st.session_state:
    st.session_state.recommendation_generated = False
if 'current_recommendation' not in st.session_state:
    st.session_state.current_recommendation = None
if 'current_user' not in st.session_state:
    st.session_state.current_user = None
if 'current_product' not in st.session_state:
    st.session_state.current_product = None
if 'current_weights' not in st.session_state:
    st.session_state.current_weights = (0.6, 0.4)

# ===============================
# 🔍 Enhanced Search Section with Dropdown Lists
# ===============================
st.markdown("### 🔍 Select User & Product")

# Prepare data for dropdowns
users_list = sorted(ratings_df['userId'].unique())
total_users = len(users_list)

# Create product list with name and ID
products_list = []
for idx, row in products_df.iterrows():
    product_display = f"{row['title'][:60]}... (ID: {row['productId']}) - {row['category']}"
    products_list.append({
        'display': product_display,
        'id': row['productId'],
        'title': row['title']
    })
total_products = len(products_list)

col1, col2 = st.columns(2)

with col1:
    st.markdown(f"**👤 Select User** ({total_users} total users)")

    # Searchable selectbox for users
    selected_user = st.selectbox(
        "Choose a user ID:",
        options=users_list,
        format_func=lambda x: f"User ID: {x}",
        key="user_selectbox",
        help="Search by typing user ID"
    )

    st.info(f"✓ Selected: User **{selected_user}**")

with col2:
    st.markdown(f"**📦 Browse Products** ({total_products} total products)")

    # Searchable selectbox for products
    selected_product_display = st.selectbox(
        "Choose a product:",
        options=[p['display'] for p in products_list],
        key="product_selectbox",
        help="Search by typing product name or ID"
    )

    # Get the actual product ID from selection
    selected_product_id = None
    for p in products_list:
        if p['display'] == selected_product_display:
            selected_product_id = p['id']
            selected_product_title = p['title']
            break

    if selected_product_id:
        st.info(f"✓ Selected: **{selected_product_title}**")

st.markdown("---")

# Get Recommendation Button
if st.button("🎯 Get Recommendation", type="primary", use_container_width=True):
    with st.spinner(f"🔄 Generating recommendation for User {selected_user} and Product {selected_product_id}..."):
        result, a_cf, a_cbf = recommend_single_product(selected_user, selected_product_id)
        st.session_state.current_recommendation = result
        st.session_state.current_user = selected_user
        st.session_state.current_product = selected_product_id
        st.session_state.current_weights = (a_cf, a_cbf)
        st.session_state.recommendation_generated = True
        st.success(f"✅ Successfully generated recommendation!")

# Display recommendation if it exists
if st.session_state.recommendation_generated and st.session_state.current_recommendation is not None:
    row = st.session_state.current_recommendation
    user = st.session_state.current_user
    a_cf, a_cbf = st.session_state.current_weights

    st.markdown("---")
    st.subheader(f"🎯 Recommendation for User {user}")
    st.caption(f"🔧 Model Weights → Collaborative Filtering: {a_cf:.2f} | Content-Based: {a_cbf:.2f}")

    st.markdown("---")

    with st.container():
        st.markdown(f"### {row['title']}")
        st.caption(f"🆔 Product ID: {row['productId']} | 📦 Category: {row['category']}")

        # Score metrics
        metric_col1, metric_col2, metric_col3 = st.columns(3)
        with metric_col1:
            st.metric("🎯 Hybrid Score", f"{row['hybrid_score']:.3f}")
        with metric_col2:
            st.metric("👥 CF Score", f"{row['cf_raw']:.3f}")
        with metric_col3:
            st.metric("📄 CBF Similarity", f"{row['cbf_sim']:.3f}")

        # Explanation
        st.markdown("**💡 Why this recommendation?**")
        for reason in row["reasons"]:
            st.markdown(f"- {reason}")

        # Description
        with st.expander("📝 View Full Description"):
            st.write(row["description"])

        # FEEDBACK SECTION
        FEEDBACK_PATH = "/content/recommender_app/models/feedback.csv"
        os.makedirs(os.path.dirname(FEEDBACK_PATH), exist_ok=True)

        def append_feedback_row(row_dict):
            header_needed = not os.path.exists(FEEDBACK_PATH)
            df_fb = pd.DataFrame([row_dict])
            try:
                df_fb.to_csv(FEEDBACK_PATH, index=False, header=header_needed, mode="a")
            except Exception:
                with open(FEEDBACK_PATH, "a") as f:
                    if header_needed:
                        f.write(",".join(df_fb.columns.tolist()) + "\n")
                    f.write(",".join(map(str, list(row_dict.values()))) + "\n")

        pid = row["productId"]

        # State keys
        state_like_key = f"liked_{user}_{pid}"
        state_dislike_key = f"disliked_{user}_{pid}"

        # Initialize
        if state_like_key not in st.session_state:
            st.session_state[state_like_key] = False
        if state_dislike_key not in st.session_state:
            st.session_state[state_dislike_key] = False

        # Callbacks
        def handle_like():
            fb = {
                "timestamp": datetime.utcnow().isoformat(),
                "userId": user,
                "productId": pid,
                "feedback_score": 1
            }
            append_feedback_row(fb)
            st.session_state[state_like_key] = True
            st.session_state[state_dislike_key] = False

        def handle_dislike():
            fb = {
                "timestamp": datetime.utcnow().isoformat(),
                "userId": user,
                "productId": pid,
                "feedback_score": 0
            }
            append_feedback_row(fb)
            st.session_state[state_dislike_key] = True
            st.session_state[state_like_key] = False

        def handle_undo_like():
            st.session_state[state_like_key] = False

        def handle_undo_dislike():
            st.session_state[state_dislike_key] = False

        # Display buttons
        st.markdown("**📢 Your Feedback:**")
        fb_col1, fb_col2, fb_col3 = st.columns([1, 1, 2])

        with fb_col1:
            if not st.session_state[state_like_key]:
                if st.button("👍 Like", key=f"like_{pid}"):
                    handle_like()
                    st.rerun()
            else:
                st.success("✓ Liked!")
                if st.button("↩️ Undo", key=f"undo_like_{pid}"):
                    handle_undo_like()
                    st.rerun()

        with fb_col2:
            if not st.session_state[state_dislike_key]:
                if st.button("👎 Dislike", key=f"dislike_{pid}"):
                    handle_dislike()
                    st.rerun()
            else:
                st.error("✗ Disliked!")
                if st.button("↩️ Undo", key=f"undo_dislike_{pid}"):
                    handle_undo_dislike()
                    st.rerun()

        with fb_col3:
            if st.session_state[state_like_key]:
                st.info("💬 Great! We'll show you more like this.")
            elif st.session_state[state_dislike_key]:
                st.info("💬 Noted! We'll adjust your recommendations.")

        st.markdown("---")

# Sidebar
st.sidebar.header("📊 System Statistics")
st.sidebar.metric("Total Products", len(products_df))
st.sidebar.metric("Total Users", len(ratings_df['userId'].unique()))
st.sidebar.metric("Total Ratings", len(ratings_df))

st.sidebar.header("📈 How It Works")
st.sidebar.markdown("""
- **Collaborative Filtering (CF)**: Learns from similar users' preferences
- **Content-Based Filtering (CBF)**: Uses product features and descriptions
- **Dynamic Weights**: Automatically adapts based on user feedback
- **Searchable Dropdowns**: Easily find any user or product
- **Real-time Feedback**: Like/dislike to improve recommendations
""")

if st.session_state.recommendation_generated:
    st.sidebar.success(f"✅ Active: Recommendation for User {st.session_state.current_user} & Product {st.session_state.current_product}")

st.sidebar.markdown("---")
st.sidebar.info("💡 **Tip**: Use the search bar in dropdowns to quickly find users or products by typing!")
PY

# Restart services
pkill -f streamlit || true
pkill -f cloudflared || true
sleep 2
nohup streamlit run /content/streamlit_recommender_app.py --server.port 8501 --server.headless true > /content/streamlit.log 2>&1 &
nohup cloudflared tunnel --url http://localhost:8501 --no-autoupdate > /content/cf.log 2>&1 &
sleep 5
grep -o "https://[a-z0-9.-]*\\.trycloudflare\\.com" -m 1 /content/cf.log || true

https://shake-knife-excited-call.trycloudflare.com
